In [46]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer


# 读取csv文件
def load_csv_data(data_path):
    df = pd.read_csv(data_path)
    return df

# 将某列的文本字符串转成列表并切词
def col_jieba_fun(series):
    # 获得字符串
    col = series[col_name]

    # 字符串变列表
    if col.startswith("[") and col.endswith("]"):
        col = json.loads(col)
    else:
        col = re.split(",|，", col)

    # 对于中文，进入jieba前不需要添加空格
    # 不过，如果是中英文混合，就必须空格了
    col_str = " ".join(col)
    col_list = jieba.lcut(col_str, cut_all=False)
    return col_list

# 对切词后的列表进行过滤
def col_jieba_filter_fun(series):

    col_list_filter = []
    # 得到tag列表
    col_list = series[col_name_jieba]
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in col_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            col_list_filter.append(tag)
    return " ".join(col_list_filter)

# 转成tfidf向量
def get_tfidf(df, col_name):
    text = df[col_name]
    
    vectorizer = TfidfVectorizer()
    vector = vectorizer.fit_transform(text)
    # 打印tfidf的大小
    print(vector.shape)
    return vector.toarray()


if __name__ == "__main__":
    print("running...")

    # csv文件和待处理的列名
    data_path = './sample_20220821_spark.csv'
    col_name = 'tags'
    col_name = 'skills'
    col_name_jieba = col_name + '_jieba'
    col_name_jieba_filter = col_name_jieba + '_filter'

    # 读取csv文件
    df = load_csv_data(data_path)
    # 空值填充
    df[col_name].fillna('[]', inplace=True)

    df[col_name_jieba] = df.apply(col_jieba_fun, axis=1)
    df[col_name_jieba_filter] = df.apply(col_jieba_filter_fun, axis=1)

    print(df[[col_name, col_name_jieba, col_name_jieba_filter]])

    tfidf_vector = get_tfidf(df, col_name_jieba_filter)
    print(tfidf_vector)
    print("all is well")


running...
                                                  skills  \
0      ["Ai","Ar","App","优化","营销","策划","运营","搭建","电商"...   
1                                     ["it规划咨询","云计算咨询"]   
2      ["VC","PPT","SQL","VBA","Kpi","Excel","C/C++",...   
3      ["Vi","Pr","App","Crm","优化","策划","营销","搭建","融资...   
4      ["PPT","Word","Excel","Visio","营销","运营","搭建","...   
...                                                  ...   
81291  O2O,Erp,Wms,Kpi,优化,运营,搭建,驾驶,电商,性能,架构,招投标,施工图,绩...   
81292                        Zcool,电商,优化,搭建,装修,页面设计,平面设计   
81293                   C1,C4D,Icon,舞蹈,美术,项目经理,视频制作,沟通能力   
81294                           制药,内科,配置,策划,美容,团队销售,生物医药   
81295  Top,Bom,优化,电气,仿真,消防,搭建,招聘,电站,迭代,普通话,系统集成,系统测试,...   

                                            skills_jieba  \
0      [Ai,  , Ar,  , App,  , 优化,  , 营销,  , 策划,  , 运营...   
1                             [it, 规划, 咨询,  , 云, 计算, 咨询]   
2      [VC,  , PPT,  , SQL,  , VBA,  , Kpi,  , Excel,...   
3      [Vi,  , Pr,  , App,  

In [47]:
tfidf_vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [55]:
# tfidf = pd.DataFrame(tfidf_vector)
from sklearn.decomposition import PCA
pca = PCA(n_components = 20)
tfidf_reduce = pca.fit_transform(tfidf_vector)
pd.DataFrame(tfidf_reduce)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.211772,0.210979,-0.168919,-0.035682,0.034876,0.009882,0.010308,0.051264,-0.066279,-0.038102,0.072379,-0.069376,-0.084426,0.036377,-0.011972,-0.043649,-0.001863,-0.005266,-0.107619,-0.019995
1,-0.086929,-0.151057,-0.060674,-0.001783,-0.042832,-0.022765,-0.021550,0.013250,-0.029863,0.000704,0.070274,0.018256,0.029005,-0.041007,-0.015188,-0.028958,-0.022509,-0.021716,0.008741,0.034069
2,-0.068580,-0.016301,-0.085728,0.134399,0.121062,0.059459,0.084901,0.050056,-0.014123,0.009099,-0.051357,0.009936,-0.000872,0.060493,0.004771,-0.081510,0.063759,0.012498,-0.040326,-0.102420
3,0.366407,0.163204,0.024744,0.030895,-0.092778,-0.074874,-0.006399,0.052054,0.053621,-0.180935,-0.130252,0.038250,-0.091291,0.041114,0.068116,-0.147966,0.062594,-0.085827,-0.024255,-0.077127
4,0.225423,-0.015059,0.064585,0.109513,0.061570,-0.105434,0.045500,-0.078191,0.153913,0.151012,-0.163199,-0.005738,0.085724,0.003160,-0.012128,-0.112014,0.054300,-0.041600,-0.037979,0.049670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.183654,0.055653,0.221114,0.142240,-0.088944,0.034020,0.108852,-0.030610,0.010004,0.009413,-0.005930,-0.151554,0.103576,-0.080771,-0.051575,-0.022719,-0.082289,-0.033170,-0.024347,0.024220
81292,-0.048168,-0.047278,-0.105356,0.013289,-0.046746,0.022862,-0.108955,0.000125,0.060618,-0.119248,-0.019166,0.061937,0.129278,0.020476,0.087054,-0.058627,-0.069399,0.026152,-0.014667,-0.015316
81293,-0.069837,-0.136389,-0.124156,0.000966,-0.037726,0.005299,-0.032903,-0.020930,0.068968,0.016815,0.018868,0.048775,-0.052668,-0.147503,0.213621,0.131090,-0.034699,-0.019468,0.010423,0.048310
81294,0.012270,-0.133919,-0.070024,-0.062740,-0.107637,-0.048481,0.026673,-0.002948,0.005562,-0.040683,0.040581,0.081514,-0.105620,0.013502,0.023728,-0.109958,-0.016195,-0.031579,0.039728,-0.026655


In [56]:
type(tfidf_reduce)

numpy.ndarray

In [48]:
tfidf_vector.shape

(81296, 3009)

In [49]:
type(tfidf_vector)

numpy.ndarray

In [43]:
import json
import jieba
import re
import numpy as np
import pandas as pd

sample_20220821 = pd.read_csv('./sample_20220821_spark.csv')

def tags_jieba_fun(series):
    tags = series["tags"]

    if tags.startswith("[") and tags.endswith("]"):
        tags = json.loads(tags)
    else:
        tags = re.split(",|，", tags)
    
    tags_str = " ".join(tags)
    tags_list = jieba.lcut(tags_str, cut_all=False)
    return " ".join(tags_list)

df1 = sample_20220821
# 空值填充
df1['tags'].fillna('[]', inplace=True)
df1['tags_jieba'] = df1.apply(tags_jieba_fun, axis=1)
df1


,Unnamed: 0,cv_id,jd_id,tags,skills,tags_jieba
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,"[""市场策划"",""管理"",""推广"",""营销"",""品牌推广""]","[""Ai"",""Ar"",""App"",""优化"",""营销"",""策划"",""运营"",""搭建"",""电商""...",市场策划 管理 推广 营销 品牌 推广
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,"[""销售"",""团队管理"",""市场推广"",""AI"",""负责管理""]","[""it规划咨询"",""云计算咨询""]",销售 团队 管理 市场推广 AI 负责管理
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,"[""投资"",""临床"",""生物"",""生物医药""]","[""VC"",""PPT"",""SQL"",""VBA"",""Kpi"",""Excel"",""C/C++"",...",投资 临床 生物 生物医药
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,"[""运营"",""产品运营"",""内容运营"",""美工"",""互联网产品""]","[""Vi"",""Pr"",""App"",""Crm"",""优化"",""策划"",""营销"",""搭建"",""融资...",运营 产品 运营 内容 运营 美工 互联网 产品
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,"[""架构"",""数字化"",""管理"",""云计算"",""区块链""]","[""PPT"",""Word"",""Excel"",""Visio"",""营销"",""运营"",""搭建"",""...",架构 数字化 管理 云 计算 区块 链
...,...,...,...,...,...,...
81291,81291,8a69f60582b081f50182b92ebdab1e84,8a69f62f82926a860182a0d7cd7d28bb,"[""仓储"",""物流"",""管理"",""沟通能力"",""软件""]","O2O,Erp,Wms,Kpi,优化,运营,搭建,驾驶,电商,性能,架构,招投标,施工图,绩...",仓储 物流 管理 沟通 能力 软件
81292,81292,8a69f61382b080760182b2fb552a0978,8a69f62f82926a860182af1bf161226d,"[""Photoshop"",""Coredraw"",""AI"",""3D工业软件"",""软件""]","Zcool,电商,优化,搭建,装修,页面设计,平面设计",Photoshop Coredraw AI 3D 工业 软件 软件
81293,81293,8a69f68b828a2cd901828f891bd10be1,8a69c4d6826e58640182804931425820,"[""游戏"",""动画设计"",""审核"",""美术"",""软件""]","C1,C4D,Icon,舞蹈,美术,项目经理,视频制作,沟通能力",游戏 动画设计 审核 美术 软件
81294,81294,8a69f68b828a2cd901828fc2ff6a4885,8a69c4d6826e5864018285e46d3f17d7,"[""沟通能力"",""归档"",""法规"",""评审"",""产品开发""]","制药,内科,配置,策划,美容,团队销售,生物医药",沟通 能力 归档 法规 评审 产品开发


In [22]:
from zhconv import convert

def tags_jieba_filter_fun(series):

    tgas_list_filter = []
    # 得到tag列表
    tags_list = series["tags_jieba"].split(" ")
    # print(tags_list[0])

    pun_masks_english = [",", ".", "/", "[", "]", "{", "}", "(", ")", ":", "*", "#", "!", " ", "\"", "\\"]
    pun_masks_chinese = ["，", "。", "、", "（", "）", "：", "！", "”", "“"]
    pun_masks = pun_masks_english + pun_masks_chinese

    # 过滤
    for tag in tags_list:
        # 转中文简体
        tag = convert(tag, "zh-hans")
        # 转英文小写
        tag = tag.lower()

        # 过滤数字
        if tag.isdigit():
            continue
        
        # 过滤单个字符
        if len(tag) <= 1:
            continue
        
        # 过滤标点
        flag = 1
        for pun in pun_masks:
            if pun in tag:
                flag = 0
                break
        if flag == 1:
            tgas_list_filter.append(tag)
    return " ".join(tgas_list_filter)
    
df1['tags_jieba_filter'] = df1.apply(tags_jieba_filter_fun, axis=1)
df1


,Unnamed: 0,cv_id,jd_id,tags,skills,tags_jieba,tags_jieba_filter
0,0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,"[""市场策划"",""管理"",""推广"",""营销"",""品牌推广""]","[""Ai"",""Ar"",""App"",""优化"",""营销"",""策划"",""运营"",""搭建"",""电商""...",市场策划 管理 推广 营销 品牌 推广,市场策划 管理 推广 营销 品牌 推广
1,1,2c9207157bf91042017c1040330d04d3,2c9207157bcbb76b017bec76efca1ca5,"[""销售"",""团队管理"",""市场推广"",""AI"",""负责管理""]","[""it规划咨询"",""云计算咨询""]",销售 团队 管理 市场推广 AI 负责管理,销售 团队 管理 市场推广 ai 负责管理
2,2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,"[""投资"",""临床"",""生物"",""生物医药""]","[""VC"",""PPT"",""SQL"",""VBA"",""Kpi"",""Excel"",""C/C++"",...",投资 临床 生物 生物医药,投资 临床 生物 生物医药
3,3,2c9207157bf91042017c62fce2583411,2c9207157b9b9199017bca11fd3d3f11,"[""运营"",""产品运营"",""内容运营"",""美工"",""互联网产品""]","[""Vi"",""Pr"",""App"",""Crm"",""优化"",""策划"",""营销"",""搭建"",""融资...",运营 产品 运营 内容 运营 美工 互联网 产品,运营 产品 运营 内容 运营 美工 互联网 产品
4,4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,"[""架构"",""数字化"",""管理"",""云计算"",""区块链""]","[""PPT"",""Word"",""Excel"",""Visio"",""营销"",""运营"",""搭建"",""...",架构 数字化 管理 云 计算 区块 链,架构 数字化 管理 计算 区块
...,...,...,...,...,...,...,...
81291,81291,8a69f60582b081f50182b92ebdab1e84,8a69f62f82926a860182a0d7cd7d28bb,"[""仓储"",""物流"",""管理"",""沟通能力"",""软件""]","O2O,Erp,Wms,Kpi,优化,运营,搭建,驾驶,电商,性能,架构,招投标,施工图,绩...",仓储 物流 管理 沟通 能力 软件,仓储 物流 管理 沟通 能力 软件
81292,81292,8a69f61382b080760182b2fb552a0978,8a69f62f82926a860182af1bf161226d,"[""Photoshop"",""Coredraw"",""AI"",""3D工业软件"",""软件""]","Zcool,电商,优化,搭建,装修,页面设计,平面设计",Photoshop Coredraw AI 3D 工业 软件 软件,photoshop coredraw ai 3d 工业 软件 软件
81293,81293,8a69f68b828a2cd901828f891bd10be1,8a69c4d6826e58640182804931425820,"[""游戏"",""动画设计"",""审核"",""美术"",""软件""]","C1,C4D,Icon,舞蹈,美术,项目经理,视频制作,沟通能力",游戏 动画设计 审核 美术 软件,游戏 动画设计 审核 美术 软件
81294,81294,8a69f68b828a2cd901828fc2ff6a4885,8a69c4d6826e5864018285e46d3f17d7,"[""沟通能力"",""归档"",""法规"",""评审"",""产品开发""]","制药,内科,配置,策划,美容,团队销售,生物医药",沟通 能力 归档 法规 评审 产品开发,沟通 能力 归档 法规 评审 产品开发


In [23]:
df1['tags_jieba_filter']

# 将新生成的列tags_jie保存下来。可以发现需要进一步过滤。
with open('./tags_jieba_filter_temp', "w") as fo:
    for line in df1['tags_jieba_filter']:
        fo.write(line + '\n')


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 文件列表
# text = ["The quick brown fox jumped over the lazy dog.",
#         "The dog.",
#         "The fox"]
text = df1['tags_jieba_filter']
# 建立transform
vectorizer = TfidfVectorizer()
# 分词，建立词汇表
vectorizer.fit(text)

# 输出结果
#print(vectorizer.vocabulary_)
#print(vectorizer.idf_)

with open("./word_idf_temp", "w") as fo:
    for word, index in vectorizer.vocabulary_.items():
        fo.write("{}\t{}\n".format(word, vectorizer.idf_[index]))
        #print(word, vectorizer.idf_[index])
    

# 词典的大小
print(len(vectorizer.vocabulary_))
# 编码文件
vector = vectorizer.transform(text)
# 输出编码结果
print(vector.shape)
print(vector.toarray())


2393
(81296, 2393)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [35]:
print(type(vector.toarray()))
tfidf = pd.DataFrame(vector.toarray())
tfidf

<class 'numpy.ndarray'>


,0,1,2,3,4,5,6,7,8,9,...,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81292,0.0,0.0,0.0,0.0,0.402959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81293,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
tfidf['sum'] = tfidf.apply(sum, axis=1)
tfidf

,0,1,2,3,4,5,6,7,8,9,...,2384,2385,2386,2387,2388,2389,2390,2391,2392,sum
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.993638
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.522898
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.923468
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.343408
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.166106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.316813
81292,0.0,0.0,0.0,0.0,0.402959,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.799790
81293,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.924344
81294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.665284


In [33]:
for index, value in enumerate(tfidf.iloc[1]):
    if value:
        print(index, value)


23 0.3980843066467165
1053 0.3160548767987825
1236 0.48180611834061005
1907 0.16419906821926017
2159 0.6460554138718986
2309 0.2552491570712822
2393 4.5228978818971


In [34]:
# 市场策划 管理 推广 营销 品牌
# 销售 团队 管理 市场推广 ai 负责管理
words = ['市场策划', '管理', '推广', '营销', '品牌']
words = '销售 团队 管理 市场推广 ai 负责管理'.split(" ")
for word in words:
    print(vectorizer.vocabulary_[word])

2309
1053
1907
1236
23
2159


In [ ]:
'''
接下来工作
1.切词后做一些过滤操作，比如去除数字、单个字符、标点等
2.降维
'''

In [39]:
# 对数据进行PCA降维
from sklearn.decomposition import PCA
pca = PCA(n_components = 20)
tfidf_reduce = pca.fit_transform(tfidf)
pd.DataFrame(tfidf_reduce)
# 对降维后的结果进行测评

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.078512,0.069618,-0.017895,0.105482,-0.015283,0.007682,-0.014454,-0.085960,-0.070056,-0.048234,0.071195,0.237524,-0.119979,0.008440,-0.013928,-0.024121,0.008485,0.189364,0.016720,0.008770
1,0.038504,0.199545,-0.020085,0.144840,0.041053,-0.043275,0.008736,0.017601,-0.036109,-0.094365,0.062546,-0.097310,-0.069673,-0.088297,-0.094070,0.055580,0.046753,-0.074511,-0.046563,0.023351
2,-0.087994,0.017962,-0.061977,0.007464,-0.077803,-0.044911,-0.014456,-0.003622,-0.078271,-0.027265,-0.073952,0.031800,0.071627,0.037743,-0.085895,-0.033185,-0.079899,-0.014100,0.002412,-0.019882
3,0.342511,-0.210294,-0.193566,-0.068237,-0.016399,0.055897,0.005739,0.020350,-0.115034,-0.005907,0.008783,-0.129218,-0.053976,0.303977,0.047035,-0.057617,-0.041459,-0.175098,0.083127,-0.012061
4,-0.075887,0.047373,-0.040878,-0.032711,0.093098,-0.033183,0.029436,-0.013971,0.022257,0.008198,0.030214,-0.017722,-0.053762,0.033846,-0.055365,0.016291,-0.023274,0.049182,-0.038961,-0.102916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81291,0.004222,0.001419,0.291622,0.001515,0.003247,-0.070986,0.029892,0.018149,0.072668,0.135787,-0.071464,-0.023122,-0.031085,0.050931,-0.060297,-0.015337,0.033086,0.016529,-0.043093,0.072987
81292,-0.116039,0.012818,-0.028575,0.047633,-0.030192,-0.001297,0.012773,0.034465,0.091745,0.181915,-0.113946,-0.018592,-0.072005,0.052709,-0.055076,-0.081392,0.037147,0.025328,-0.037035,0.082239
81293,-0.091066,-0.021294,-0.044186,0.019165,-0.137924,-0.008029,0.048419,-0.079589,0.296037,-0.007221,-0.079076,-0.022675,0.021589,0.058957,0.058629,-0.067919,0.012645,0.012262,-0.068992,0.061169
81294,-0.049417,-0.081452,0.239921,0.004248,-0.067485,-0.025567,-0.008017,0.016983,-0.047561,0.020535,-0.032819,0.000908,0.037195,0.033035,-0.037894,0.034740,0.023072,-0.005460,-0.038729,-0.016371
